GridSearch (choose parameter to try, lr, dropout)
early stopping

Deactivating callbacks can be especially useful when you do a parameter search (say with sklearn GridSearchCV). If, for instance, you use a callback for learning rate scheduling (e.g. via LRScheduler) and want to test its usefulness, you can compare the performance once with and once without the callback.


In [1]:
import os
import numpy as np
import scipy.io

In [2]:
mods = ['BPSK', 'DQPSK', 'GFSK', 'GMSK', 'OQPSK',
        'PAM4', 'PAM8', 'PSK8', 'QAM16', 'QAM64', 'QPSK']
class_num = len(mods)

In [3]:
def import_from_mat(mods, fpath):

    # import data of mat format
    data = scipy.io.loadmat(fpath)
    features = []
    labels = []
    for mod in mods:
        real = np.array(data[mod].real)
        imag = np.array(data[mod].imag)
        signal = np.concatenate([real, imag], axis=1)
        features.append(signal)
        labels.append(mods.index(mod) * np.ones([signal.shape[0], 1]))

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [4]:
features, labels = import_from_mat(mods, 
                        "D:/Archive/0006/"
                        "batch100000_symbols128_sps8_baud1_snr5.dat")

In [5]:
features = features.astype(np.float32)
labels = labels.astype(np.int64)[:100000]
X = features[:100000]
y = labels[:100000].reshape(-1)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Discriminator(nn.Module):
    """Define the model"""

    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(2, 256, 3, padding=1),  # batch, 256, 1024
            nn.BatchNorm1d(256),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 80, 3, padding=1),  # batch, 80, 1024
            nn.BatchNorm1d(80),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(80 * 1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.6)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, class_num),
            nn.ReLU()
        )
        
    def forward(self, x, **kwargs):
        x = x.reshape((x.size(0), 2, -1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [8]:
from skorch import NeuralNetClassifier
from skorch.callbacks import ProgressBar

In [9]:
bar = ProgressBar

net = NeuralNetClassifier(
    Discriminator,
    max_epochs=20,
    lr=0.01,
    device='cuda',
    callbacks = [bar],
    iterator_train__shuffle=True,
    iterator_valid__shuffle=True
)

In [10]:
net.fit(X, y)

A Jupyter Widget

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.1366       1.0000        0.0081  32.0291


A Jupyter Widget

      2        0.0102       1.0000        0.0034  30.1109


A Jupyter Widget

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Discriminator(
    (conv1): Sequential(
      (0): Conv1d(2, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1d(256, 80, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (fc1): Sequential(
      (0): Linear(in_features=81920, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.6)
    )
    (fc2): Sequential(
      (0): Linear(in_features=256, out_features=11, bias=True)
      (1): ReLU()
    )
  ),
)

In [11]:
net.history

[{'batches': [{'train_batch_size': 128, 'train_loss': 2.458001136779785},
   {'train_batch_size': 128, 'train_loss': 2.4795424938201904},
   {'train_batch_size': 128, 'train_loss': 2.447953462600708},
   {'train_batch_size': 128, 'train_loss': 2.4141764640808105},
   {'train_batch_size': 128, 'train_loss': 2.406116008758545},
   {'train_batch_size': 128, 'train_loss': 2.3246922492980957},
   {'train_batch_size': 128, 'train_loss': 2.2369179725646973},
   {'train_batch_size': 128, 'train_loss': 2.2070212364196777},
   {'train_batch_size': 128, 'train_loss': 2.1116485595703125},
   {'train_batch_size': 128, 'train_loss': 2.036665201187134},
   {'train_batch_size': 128, 'train_loss': 1.9238135814666748},
   {'train_batch_size': 128, 'train_loss': 1.8388071060180664},
   {'train_batch_size': 128, 'train_loss': 1.7664684057235718},
   {'train_batch_size': 128, 'train_loss': 1.6460256576538086},
   {'train_batch_size': 128, 'train_loss': 1.57659912109375},
   {'train_batch_size': 128, 'train

320/|/ 41%|| 320/782 [00:30<00:20, 22.60it/s, train_loss=0.00487, valid_loss=0.00267, refresh=0]                       